<a href="https://colab.research.google.com/github/noahgift/mlrun-ideas-may-2022/blob/main/mlrun_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup and Install

Install mlrun

In [7]:
!pip install mlrun

## Download Data

Download data here:  https://docs.mlrun.org/en/latest/quick-start/quick-start.html#define-mlrun-project-and-ml-functions

In [8]:
!wget https://docs.mlrun.org/en/latest/_downloads/e7832d14335f1114697e6896c53e3ae0/gen_breast_cancer.py
!wget https://docs.mlrun.org/en/latest/_downloads/c3b0786b7e6d2e742259062e9ccd07ee/trainer.py
!wget https://docs.mlrun.org/en/latest/_downloads/6087b38f0719359deb30c3b3c16adac5/serving.py

--2022-05-17 10:58:51--  https://docs.mlrun.org/en/latest/_downloads/e7832d14335f1114697e6896c53e3ae0/gen_breast_cancer.py
Resolving docs.mlrun.org (docs.mlrun.org)... 104.17.33.82, 104.17.32.82, 2606:4700::6811:2052, ...
Connecting to docs.mlrun.org (docs.mlrun.org)|104.17.33.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 961 [text/x-python]
Saving to: ‘gen_breast_cancer.py.1’

gen_breast_cancer.p 100%[===================>]     961  --.-KB/s    in 0s      

2022-05-17 10:58:51 (21.4 MB/s) - ‘gen_breast_cancer.py.1’ saved [961/961]

--2022-05-17 10:58:51--  https://docs.mlrun.org/en/latest/_downloads/c3b0786b7e6d2e742259062e9ccd07ee/trainer.py
Resolving docs.mlrun.org (docs.mlrun.org)... 104.17.33.82, 104.17.32.82, 2606:4700::6811:2052, ...
Connecting to docs.mlrun.org (docs.mlrun.org)|104.17.33.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1030 (1.0K) [text/x-python]
Saving to: ‘trainer.py.1’

trainer.py.1        100%[======

## MLRun Quickstart

In [9]:
import mlrun

# check if we are attached to k8s for running remote (container) jobs
no_k8s = False if mlrun.mlconf.namespace else True

### Define MLRun project and ML functions

In [11]:
project = mlrun.new_project("breast-cancer", "./", user_project=True, init_git=True)
project.set_function("gen_breast_cancer.py", "gen-breast-cancer", image="mlrun/mlrun")
project.set_function("trainer.py", "trainer", 
                     handler="train", image="mlrun/mlrun")
project.set_function("serving.py", "serving", image="mlrun/mlrun", kind="serving")
#project.save()

In [12]:
gen_data_run = project.run_function("gen-breast-cancer", params={"format": "csv"}, local=True)

> 2022-05-17 10:59:50,038 [warning] artifact path is not defined or is local, artifacts will not be visible in the UI
> 2022-05-17 10:59:50,043 [info] starting run gen-breast-cancer uid=af6c44f53a7d44c4ba1e3a67c4f41351 DB=./
> 2022-05-17 10:59:50,062 [info] handler was not provided running main (./gen_breast_cancer.py)
> 2022-05-17 10:59:53,231 [info] logging run results to: ./
> 2022-05-17 10:59:53,252 [info] saving breast cancer dataframe


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
breast-cancer-root,...f41351,0,May 17 10:59:53,completed,gen-breast-cancer,kind=owner=roothost=e55a96a05077,,format=csv,label_column=label,dataset


> 2022-05-17 10:59:54,216 [info] run executed, status=completed


Invoke via CLI

In [13]:
!mlrun run -f gen-breast-cancer --local

> 2022-05-17 11:01:12,025 [warning] artifact path is not defined or is local, artifacts will not be visible in the UI
> 2022-05-17 11:01:12,025 [info] starting run gen-breast-cancer uid=d25500116b004fc49f171b526650a96f DB=./
> 2022-05-17 11:01:12,035 [info] handler was not provided running main (./gen_breast_cancer.py)
> 2022-05-17 11:01:15,051 [info] logging run results to: ./
> 2022-05-17 11:01:15,066 [info] saving breast cancer dataframe
to track results use the CLI:
info: mlrun get run d25500116b004fc49f171b526650a96f -p breast-cancer-root
logs: mlrun logs d25500116b004fc49f171b526650a96f -p breast-cancer-root
> 2022-05-17 11:01:15,943 [info] run executed, status=completed


print output dataset artifact

In [14]:
gen_data_run.artifact("dataset").as_df().head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
